In [ ]:
#===========================================================================================
# ElasticSearch 텍스트 임베딩 테스트 예제
# - 문장들을  추출 요약해서 요약문장을 만들고, 요약 문장의 평균을 구하여 문장 embedding을 생성하여 ES에 인덱스에 vector 추가하고, 검색하는 예제임
# - 말뭉치는 ai_hub에 원천말뭉치인 ts1 말뭉치에 tilte, content를 추출하여, content 요약문과 title에 대해 각각 vector를 만들어서 ES 인덱스로 추가하는 예시임.
#
# => 대규모 웹데이터 기반 한국어 말뭉치 데이터 
# 말뭉치 출처: https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=624

# -여기서는 elasticsearch 7.17.3 때를 기준으로 설명함.
# -** 따라서 elasticsearch python 모듈도 7.17.3 을 설치해야 함
# - elasticsearch 모듈 8.x 부터는 구문의 많이 변경되었음.
# - 예 : index 생성:  body로 모든 변수들를 지정하는 데시, 명시적으로 모든 변수들을 최상으로 지정해 줘야함.
# => 참고: https://towardsdatascience.com/important-syntax-updates-of-elasticsearch-8-in-python-4423c5938b17   

# =>ElasticSearch 7.3.0 버전부터는 cosine similarity 검색을 지원한다.
# => 데이터로 고차원벡터를 집어넣고, 벡터형식의 데이터를 쿼리(검색어)로 하여 코사인 유사도를 측정하여 가장 유사한 데이터를 찾는다.
# => 여기서는 ElasticSearch와 S-BERT를 이용함
# => ElasticSearch에 index 파일은 index_1.json /데이터 파일은 KorQuAD_v1.0_train_convert.json 참조
#
# => 참고자료 : https://skagh.tistory.com/32
#
#===========================================================================================

# sentenceTransformers 라이브러리 설치
#!pip install -U sentence-transformers

# elasticsearch 서버 접속 모듈 설치
# !pip install elasticsearch==7.17

# 한국어 문장 분리기(kss) 설치
#!pip install kss

# 추출 요약 설치
#!pip install bert-extractive-summarizer

In [1]:
import torch
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.cross_encoder import CrossEncoder

import kss
import numpy as np
import json
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from tqdm.notebook import tqdm

from elasticsearch import Elasticsearch
from elasticsearch import helpers


# FutureWarning 제거
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) 

import sys
sys.path.append('..')
from myutils import seed_everything, GPU_info, getListOfFiles
device = GPU_info()
#device = torch.device('cpu')

#------------------------------------------------------------------------------------
# 0. param 설정
#------------------------------------------------------------------------------------
seed = 111
query_num = 500             # 쿼리 최대 갯수: KorQuAD_v1.0_dev.json 최대값은 5533개임, 0이면 모든 5533개 쿼리함.
search_k = 5                # FAISS 검색시, 검색 계수(5=쿼리와 가장 근접한 5개 결과값을 반환함)
avg_num = 1                 # 쿼리에 대해 sub 문장들중 최대 scorce를 갖는 문장을 몇개 찾고 평균낼지.(3=쿼리에 가장 유사한 sub문장 3개를 찾고 평균을 냄)
faiss_index_method = 0      # 0= Cosine Similarity 적용(IndexFlatIP 사용), 1= Euclidean Distance 적용(IndexFlatL2 사용)

# 토큰 임베딩 관련 param
IS_EMBED_DIVIDE = True      #여기서는 토큰 임베딩은 True고정, True=문단의 여러 문장을, 토큰 단위로 분리후 벡터 구해서 인덱스 만듬/False=문단의 여러문장을 하나의 벡터를 구해서 인덱스 만듬.
EMBED_DIVIDE_LEN = [5,7,9]  #5 # 문장을 몇개(토큰)으로 분리할지 (7,8,10) 일때 성능 좋음=>50.8%, (5,7,9) 일때 차원축소 128=>41.80%(81.8%) 성능 좋음
MAX_TOKEN_LEN = 40          # 최대 몇개 token까지만 임베딩 할지

# 차원 축소 관련 param
# 차원 축소 할지, 768 값을 앞에 384 만큼만 배열 resize 시킴.  
# - 384로 줄일대 -2% 성능 저하 발생(512: -1.2%, 256: -6%)
DIM_RESIZE_METHOD = 2  # 0=차원축소 안함/1=resize 로 차원축소/2=Dense 모델 이용 차원축소
DIM_RESIZE_LEN = 128

# ONNX 모델 사용
IS_ONNX_MODEL = True        # True=onnx 모델 사용
#------------------------------------------------------------------------------------

seed_everything(seed)

# elastic 서버 접속 테스트
#es = Elasticsearch("https://192.168.0.91:9200/", verify_certs=False)
#es = Elasticsearch("http://192.168.0.130:9200/")
#es.info()


False
device: cpu


In [2]:
# 엘라스틱 서치에 인덱스 생성 하고 dense 벡터 128차원 5개 추가하는 예제임
'''
from elasticsearch import Elasticsearch
import numpy as np

# define the Elasticsearch client
es = Elasticsearch("http://192.168.0.27:9200/")

mapping='''
{
  "settings": {
    "number_of_shards": 2,
    "number_of_replicas": 1
  },
   "mappings": {
    "dynamic": "true",
    "_source": {
      "enabled": "true"
    },
    "properties": {
      "vector1": {
        "type": "dense_vector",
        "dims": 128
      },
	  "vector2": {
        "type": "dense_vector",
        "dims": 128
      },
      "vector3": {
        "type": "dense_vector",
        "dims": 128
      },
      "vector4": {
        "type": "dense_vector",
        "dims": 128
      },
      "vector5": {
        "type": "dense_vector",
        "dims": 128
      }
    }
  }
}
'''

###########################################################
# 인덱스 생성/삭제
###########################################################
## 인덱스 생성
def create_index(index, mapping=None):
    if not es.indices.exists(index=index):
        return es.indices.create(index=index ,body=mapping)
    
## 인덱스 자체 삭제
def delete_index(index):
    if es.indices.exists(index=index):
        return es.indices.delete(index=index)
    
###########################################################
# 인데스에 데이터 추가 
###########################################################
def insert(index, doc_type, body):
    return es.index(index=index, doc_type=doc_type, body=body)

# index 생성
INDEX_NAME = 'my_index_128_5-1'
res=create_index(index=INDEX_NAME, mapping=mapping)
print(res)
print('\n')

result=es.indices.get_settings(index=INDEX_NAME)
print(result)
print('\n')


# document 추가 
DOC_TYPE = '_doc'

# define the dense vector
dense_vector1 = np.random.rand(128).tolist()
dense_vector2 = np.random.rand(128).tolist()
dense_vector3 = np.random.rand(128).tolist()
dense_vector4 = np.random.rand(128).tolist()
dense_vector5 = np.random.rand(128).tolist()

doc = {
    'vector1': dense_vector1,
    'vector2': dense_vector2,
    'vector3': dense_vector3,
    'vector4': dense_vector4,
    'vector5': dense_vector5
}

res = insert(index=INDEX_NAME, doc_type=DOC_TYPE, body=doc) 
print(res)
'''

"\n\n###########################################################\n# 인덱스 생성/삭제\n###########################################################\n## 인덱스 생성\ndef create_index(index, mapping=None):\n    if not es.indices.exists(index=index):\n        return es.indices.create(index=index ,body=mapping)\n    \n## 인덱스 자체 삭제\ndef delete_index(index):\n    if es.indices.exists(index=index):\n        return es.indices.delete(index=index)\n    \n###########################################################\n# 인데스에 데이터 추가 \n###########################################################\ndef insert(index, doc_type, body):\n    return es.index(index=index, doc_type=doc_type, body=body)\n\n# index 생성\nINDEX_NAME = 'my_index_128_5-1'\nres=create_index(index=INDEX_NAME, mapping=mapping)\nprint(res)\nprint('\n')\n\nresult=es.indices.get_settings(index=INDEX_NAME)\nprint(result)\nprint('\n')\n\n\n# document 추가 \nDOC_TYPE = '_doc'\n\n# define the dense vector\ndense_vector1 = np.random.rand(128).tolist()\ndense_ve

In [2]:
#-------------------------------------------------------------------------------------
# 1. 검색모델 로딩
# => bi_encoder 모델 로딩, polling_mode 설정
#-------------------------------------------------------------------------------------
from myutils import bi_encoder, dense_model, onnx_model

# bi_encoder 모델 로딩
bi_encoder_path = "bongsoo/klue-sbert-v1"
pooling_mode = 'mean' # bert면=mean, albert면 = cls
 # 출력 임베딩 크기 지정 : 0=기본 모델 임베딩크기(768), 예:128=128 츨력임베딩 크기 
out_dimension = 128 if DIM_RESIZE_METHOD == 2 else 0 if DIM_RESIZE_METHOD != 2 else None
    
word_embedding_model1, bi_encoder1 = bi_encoder(model_path=bi_encoder_path, max_seq_len=512, do_lower_case=True, 
                                              pooling_mode=pooling_mode, out_dimension=out_dimension, device=device)
print(f'---bi_encoder---------------------------')   
print(bi_encoder1)
print(word_embedding_model1)

print(f'---dense param---------------------------')   
# 출력 값 차원 축소 지정인 경우, token_embeddings 일때는 sentencebert 라이브러리를 이용하여 dense_model 모델 추가할수 없으므로,
# 사용자정의 dense_model을 정의해서, 가중치와 bias를 bi_encoder모델에 것을 얻어와서 적용해서 차원 죽소함.
# => resize 방식 보다 성능 떨어지지만, 128일때는 더 성능이 좋음
if DIM_RESIZE_METHOD == 2:
    #-------------------------------------------------------------------------
    # 처음에는 아래 코드를 활용하여 해당 모델의 128 weight와 bias를 저장해 두어야 함.
    #state_dict = bi_encoder1.state_dict()  # bi_encoder모델이 state_dict 얻어옴
    #print(state_dict.keys())
    #dense_weight = state_dict['2.linear.weight'] # denser 모델에 bi_encoder모델이 linear.weight 적용
    #dense_bias = state_dict['2.linear.bias']     # denser 모델에 bi_encoder모델이 linear.bias 적용
    
    # 처음에  weigth, bias 파일을 저장해 둠.
    #torch.save(dense_weight, 'klue-sbert-v1-weigth.pt')
    #torch.save(dense_bias, 'klue-sbert-v1-bias.pt')
    #-------------------------------------------------------------------------
    # weigth, bias 저장해둔 파일 로딩
    dense_weight = torch.load('../embedding_sample/faiss/data/dense_weight/klue-sbert-v1-weight-128.pt')
    dense_bias = torch.load('../embedding_sample/faiss/data/dense_weight/klue-sbert-v1-bias-128.pt')

    print('*dense_weight:{}'.format(dense_weight.size()))
    print(f'*dense_bias:{dense_bias.size()}')
 
# onnx 모델 로딩
if IS_ONNX_MODEL == True:
    onnx_model_path = "bongsoo/klue-sbert-v1-onnx"#"bongsoo/klue-sbert-v1-onnx"
    onnx_tokenizer, onnx_model = onnx_model(onnx_model_path)
    print(f'---onnx_model---------------------------')
    print(onnx_model)

---bi_encoder---------------------------
SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Dense({'in_features': 768, 'out_features': 128, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
)
Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
---dense param---------------------------
*dense_weight:torch.Size([128, 768])
*dense_bias:torch.Size([128])
---onnx_model---------------------------


In [3]:
#-------------------------------------------------------------------------------------
# 안덱싱 및 검색 조건에 맞게 임베딩 처리 하는 함수 
#-------------------------------------------------------------------------------------
from myutils import embed_text, onnx_embed_text

def embedding(paragrphs:list):
    if IS_ONNX_MODEL == True:
        if IS_EMBED_DIVIDE == True: # 한 문단에 대한 40개 문장들을 토큰단위로 쪼개서 임베딩 처리함  
            #----------------------------------------------------
            # 한 문단에 대한 문장들의 토큰을 ?개씩 나누고 비교.
            # - 한 문단에 대한 문장들에 대해 [tensor(250,768), tensor(243,768), tensor(111,768),..] tensor 리스트 타입으로 벡터 생성됨.
            #----------------------------------------------------
            embeddings = onnx_embed_text(model=onnx_model, tokenizer=onnx_tokenizer, paragraphs=paragrphs) 
        else: # 한 문단에 대한 40개 문장 배열들을 한꺼번에 임베딩 처리함
            embeddings = onnx_embed_text(model=onnx_model, tokenizer=onnx_tokenizer, paragraphs=paragrphs, token_embeddings=False)
    else:
        if IS_EMBED_DIVIDE == True: # 한 문단에 대한 40개 문장들을 토큰단위로 쪼개서 임베딩 처리함  
            #----------------------------------------------------
            # 한 문단에 대한 문장들의 토큰을 ?개씩 나누고 비교.
            # - 한 문단에 대한 문장들에 대해 [tensor(250,768), tensor(243,768), tensor(111,768),..] tensor 리스트 타입으로 벡터 생성됨.
            #----------------------------------------------------
            embeddings = embed_text(model=bi_encoder1, paragraphs=paragrphs, token_embeddings=True, return_tensor=False)
        else: # 한 문단에 대한 40개 문장 배열들을 한꺼번에 임베딩 처리함
            embeddings = embed_text(model=bi_encoder1, paragraphs=paragrphs, return_tensor=False)  
            
    return embeddings

In [4]:
#-------------------------------------------------------------------------------------
# 안덱싱 처리
#-------------------------------------------------------------------------------------
from tqdm.notebook import tqdm
import kss
from myutils import embed_text, divide_arr_avg_exten, clean_text

# 인덱싱 함수 
def index_data(createindex:bool = True):
    
    if createindex == True:
        es.indices.delete(index=INDEX_NAME, ignore=[404])
        count = 0
        # 인덱스 생성
        with open(INDEX_FILE) as index_file:
            source = index_file.read().strip()
            count += 1
            #print(f'{count}:{source}') # 인덱스 구조 출력
            es.indices.create(index=INDEX_NAME, body=source)
        
    # json 파일들이 있는 폴더에 .json 파일 이름들을 얻기
    # =>DATA_FOLDER: .JSON파일들이 있는 폴더
    files = getListOfFiles(DATA_FOLDER)
    assert len(files) > 0 # files가 0이면 assert 발생
    print('*file_count: {}, file_list:{}'.format(len(files), files[0:5]))
 
    for idx, file in enumerate(tqdm(files)):
        if ".json" not in file:  #.json 파일이 아니면 합치지 않음
            continue
            
        count = 0
        docs = []
    
        # json 파일 로딩 => [SJML][text] entry만 불러옴
        json_data = json.load(open(file, "r", encoding="utf-8"))['SJML']['text']
        for data in json_data:
        #for data in json_data:
            count += 1
            doc = {} #dict 선언
            
            doc['title'] = data['title']            # 제목 설정
            doc['paragraph'] = data['content']      # 문장 담음.
                
            docs.append(doc)
            #print(f'count:{count}')
            #print(doc['title'])
            
            if count % BATCH_SIZE == 0:
                index_batch(docs)
                docs = []
                print("Indexed {} documents.".format(count))
                  
            # ** 10 개만 보냄
            #if count >= 10:
            #   break
            
        if docs:
            index_batch(docs)
            print("Indexed {} documents.".format(count))   
            
        es.indices.refresh(index=INDEX_NAME)
            
    es.indices.refresh(index=INDEX_NAME)
    #print("=== End Done indexing===")
                   

def index_batch(docs):
        
    requests = []
    
    for i, doc in enumerate(tqdm(docs)):
        title = doc['title']
        paragraph = doc['paragraph']

        sub_contexts = []
        #------------------------------------------------------------------------------------------------------------------------
        paragraph = clean_text(paragraph)  # 전처리 : (한글, 숫자, 영문, (), {}, [], %, ,,.,",')  등을 제외한 특수문자 제거
        
        # 입력 문단 길이가 999개 크면, 속도가 느려지므로 최대 999개 까지만 문자 입력 받음.
        if len(paragraph) > 999:
            #print(f'paragraph_len:{len(paragraph)}')
            paragraph = paragraph[0:999]
            
        # 입력 문단을 여러 문장들로 나눔.
        #sentences = [sentence for sentence in paragraph.split('.') if sentence != '' and len(sentence) > 10]  # '.'(마침표) 로 구분해서 sub 문장을 만듬.
        #sentences = [sentence for sentence in kss.split_sentences(paragraph) if sentence != '' and len(sentence) > 10] # kss 이용해서 sub 문장을 만듬
        
        # 최대 10개 문장만 추출함.
        sentences = []
        count = 0
        for sentence in kss.split_sentences(paragraph):
            if sentence != '' and len(sentence) > 10:
                sentences.append(sentence)
                if count > 10:
                    break
                    
        # 만약 sentences(sub 문장) 가 하나도 없으면 원본문장을 담고, 10이상이면  10개만 담음.
        sub_contexts.append([paragraph] if len(sentences) < 1 else sentences[0:10] if len(sentences) > 10 else sentences)
       
        if i < 1:
            print(sub_contexts[0])
        
        #------------------------------------------------------------------------------------------------------------------------
        # 토큰 분할 임베딩 처리
        # => sub_contexts은  1차원 리스트 임 (예:['오늘 비가 온다','오늘 눈이 온다','날씨가 좋다'])
        token_embeds = embedding(sub_contexts[0])
        #------------------------------------------------------------------------------------------------------------------------ 
        # 토큰 분할인 경우 처리 start=>           
        token_embed_arr_list = []
        tcount = 0
        # tensor(250,768) 한문장 토큰 임베딩 얻어와서, 각 ?개 토큰씩 평균을 구함.
        for token_embed in token_embeds:
            token_embed = token_embed[1:-1] # 맨앞에 [CLS]와 맨뒤에 [SEP] 토큰은 뺌
            if tcount >= MAX_TOKEN_LEN: 
                break
             
            #print(f'token_embed.shape:{token_embed.shape}')
            
            token_embed_arrs = token_embed.cpu().numpy().astype('float32')
            #print(f'token_embed_arrs:{token_embed_arrs.shape}')
            # 5,7,9 씩 자르면서 문장 토큰 평균을 구함
            token_embed_divide_arrs = divide_arr_avg_exten(embed_arr=token_embed_arrs, divide_arrs=EMBED_DIVIDE_LEN) 

             # Dense 방식으로 차원 축소 => 평균 구한후 차원 축소하는 방식이 0.6% 정도 성능 좋음
            if DIM_RESIZE_METHOD == 2:
                tmp1 = torch.Tensor(token_embed_divide_arrs)
                #tmp1 = torch.from_numpy(token_embed_divide_arrs)
                debug1 = False
                tmp2 = dense_model(embed_tensor=tmp1, out_f=DIM_RESIZE_LEN, weight=dense_weight, bias=dense_bias, debug=debug1)
                arrs = tmp2.detach().numpy().astype('float32')
            else:  
                arrs = token_embed_divide_arrs
                
            # 평균 구한 토큰들을 token_embed_arr_list 리스트에 담아둠.(50보다 크면 50개만 담음)           
            for idx, arr in enumerate(arrs):
                
                 # Resize 방식으로 차원 축소(384로 줄일대 -2% 성능 저하 발생)
                if DIM_RESIZE_METHOD == 1:
                    darr = np.resize(arr, (DIM_RESIZE_LEN,))
                else:
                    darr = arr
                    
                token_embed_arr_list.append(darr)
                tcount +=1
                if tcount >=MAX_TOKEN_LEN:
                    break

        #embeddings = np.array(token_embed_arr_list)
        #------------------------------------------------------------------------------------------------------------------------
        # ES에 문단 인덱싱 처리
        request = {}  #dict 정의
        request["rfile_name"] = title       # 제목               
        request["rfile_text"] = paragraph   # 문장
        
        request["_op_type"] = "index"        
        request["_index"] = INDEX_NAME
        
        # for문 돌면서 벡터 처리
        #print(type(token_embed_arr_list))
        #print(len(token_embed_arr_list))
        
        # vector 1~40 까지 값을 0으로 초기화 해줌.
        for i in range(MAX_TOKEN_LEN):
            if DIM_RESIZE_METHOD > 0:
                request["vector"+str(i+1)] = np.zeros((DIM_RESIZE_LEN))
            else:
                request["vector"+str(i+1)] = np.zeros((768))
            
        # vector 값들을 담음.
        for i, token_embed_arr in enumerate(token_embed_arr_list):
            request["vector"+str(i+1)] = token_embed_arr
            
        requests.append(request)
        #------------------------------------------------------------------------------------------------------------------------
                
    # batch 단위로 한꺼번에 es에 데이터 insert 시킴     
    bulk(es, requests)
        

In [ ]:
#======================================================================================
# ElasticSearch(이하:ES) 데이터 인텍싱
# - ElasticSearch(이하:ES)에 KorQuAD_v1.0_train_convert.json 파일에 vector값을 구하여 index 함
#
# => index 명 : korquad
# => index 구조 : index_1.json 파일 참조
# => BATCH_SIZE : 100 => 100개의 vector값을 구하여, 한꺼번에 ES에 인텍스 데이터를 추가함.
#======================================================================================
INDEX_NAME = 'aihub-ts1-acsampe-klue-sbert-v1-mpower10u-128d-onnx-1'  # ES 인덱스 명 (*소문자로만 지정해야 함)
INDEX_FILE = './data/mpower10u_128d.json'                 # 인덱스 구조 파일
DATA_FOLDER = '../../../data11/ai_hub/ts1/sample2/'     # 인덱스할 파일들이 있는 폴더경로 
BATCH_SIZE = 100

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.27:9200/")
print(es.info())

# 2. index 처리
index_data(False)

{'name': 'node-1', 'cluster_name': 'mpower', 'cluster_uuid': '2TN-XaLkReC_BmRoReVUfw', 'version': {'number': '7.17.3', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '5ad023604c8d7416c9eb6c0eadb62b14e766caff', 'build_date': '2022-04-19T08:11:19.070913226Z', 'build_snapshot': False, 'lucene_version': '8.11.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}
*file_count: 10, file_list:['../../../data11/ai_hub/ts1/sample2/BWSC217000049039.json', '../../../data11/ai_hub/ts1/sample2/BWSC217000049056.json', '../../../data11/ai_hub/ts1/sample2/BWSC217000049067.json', '../../../data11/ai_hub/ts1/sample2/BWSC217000049075.json', '../../../data11/ai_hub/ts1/sample2/BWSC217000049087.json']


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/95 [00:00<?, ?it/s]

[Kss]: Because there's no supported C++ morpheme analyzer, Kss will take pecab as a backend. :D
For your information, Kss also supports mecab backend.
We recommend you to install mecab or konlpy.tag.Mecab for faster execution of Kss.
Please refer to following web sites for details:
- mecab: https://cleancode-ws.tistory.com/97
- konlpy.tag.Mecab: https://uwgdqo.tistory.com/363



["(이름) 장군의 명량대첩을 소재로 한 대작 영화 '명량'이 '아바타'를 제치고 한국 역대 영화 흥행 1위를 기록하면서 환하게 웃는 통신사가 있다.", '아닌 업계 유일하게 영화 제작에 투자한 sk브로드밴드다. . .', "영화 '명량'이 개봉 15일 만에 1200만명을 돌파하고 급기야 '아바타'가 세운 역대 흥행기록(1330만)마저 갈아치우면서 덩달아 투자사들도 '초대형 대박'을 앞둔 셈이다.. .", "'명량' 총 제작비 189억원 가운데 sk브로드밴드가 콘텐츠펀드를 통해 투자한 금액은 수십억원에 달한다.", '영화 유통이 종료돼봐야 수 있지만, 투자 수익금은 원금의 최소 두배 이상을 가뿐히 넘길 것으로 점쳐지고 있다. . .', 'sk브로드밴드의 지난 2분기 당기 순이익이 8억원인 점을 감안한다면, 적잖은 영업외 수익을 얻게 되는 셈이다.', '다만 콘텐츠펀드가 다른 영화작품에도 투자되고 있어, 연말 결산 펀드투자 수익금은 차이가 있을 수 있다.. .', "'명량' 흥행돌풍에 따른 홍보효과도 만만치 않다.", '영화가 상영될 때마다 오프닝과 엔딩 크레딧에 회사로고와 (이름) 사장 이름이 올라간다.', '메인포스터 하단에도 회사 로고가 들어가 있다. . .']
Indexed 95 documents.


...

['오늘(6일)은 전국이 대체로 맑겠으나 해기차에 의해 새벽부터 오전 사이, 전남과 제주도에는 눈이 오다가 밤에 대부분 그치겠다. . .', '낮 동안 소강상태를 보이는 곳도 있겠으며 특히 전남(이름)은 오늘 오후까지, 제주도산지와 울릉도는 모레까지 눈이 이어지면서 많은 눈이 쌓이는 곳이 있겠다.', '내린 눈이 얼어 도로가 미끄러운 곳이 있겠으니 시설물 관리와 교통안전에 유의하기 바란다고 기상청은 밝혔다. . .', '기상청은 또 이번 추위가 당분간 이어지면서 중부지방과 일부 남부내륙에는 아침 기온이 영하 15도 이하로, 그밖의 지역은 영하 10도 이하로 떨어지는 곳이 있겠고 낮 기온도 영하권에 머물러 매우 춥겠으니 가축의 동사, 비닐하우스 작물의 동해, 수도관 동파 등 피해가 없도록 각별히 유의하라고 밝혔다. . .', '6일 아침 기온은 서울경기 14도 경남 9도 경북 10도 전남 9도 전북 11도 충남 12도 충북 12도 (이름)서 18도 (이름)동 11도 제주 1도 등으로 예상된다.. .', '낮 기온은 서울경기 6도 경남 0도 경북 2도 전남 3도 전북 3도 충남 4도 충북 5 (이름)서 5도 (이름)동 3도 제주 1도 등으로 전망된다.']
Indexed 99 documents.


  0%|          | 0/95 [00:00<?, ?it/s]

["613 전국동시 지방선거에 '국민투표로또'가 돌아왔다.", '국민투표로또는 (이름) 작가가 jtbc 썰전에서 투표율 증진의 방법으로 제안했던 것을 스타트업 개발자 및 디자이너가 실제 서비스로 만든 것. . .', "지난 4일 새롭게 문을 연 '국민투표로또' 홈페이지에는 참여방법이 자세히 나온다.", '국민투표로또는 후원금을 바탕으로 운영되며 후원금 가운데 운영비를 제외한 나머지 금액 모두 당첨금으로 사용된다. . .', '1, 2, 3등은 각각 후원금의 50%(최대 500만원), 20%(최대 200만원), 10%(최대 100만원)를 받는다.', '남은 돈은 5만원씩 다수의 4등 당첨자들에게 돌아간다.. .', '국민투표로또는 인증사진뿐만 아니라 투표소를 배경으로 한 사진, 투표 참여를 (이름)는 사진 등 선거를 즐기는 나만의 사진으로 응모 가능하다.. .', '응모방법은 사전투표 기간(6월 89일)과 투표일(6월13일)에 국민투표로또 홈페이지에 접속해 카카오톡 로그인 후 사진과 연락처를 입력하면 된다. . .', '투표 인증사진은 작년 1월 개정된 선거법으로 인해 투표 당일 특정 후보자 벽보 앞에서 지지나 반대 표시를 한 인증사진도 가능하다.', '손가락으로 엄지를 치켜세우거나 브이(v)도 할 수 있다.']
Indexed 95 documents.


  0%|          | 0/96 [00:00<?, ?it/s]

['밤사이 수도권에 많은 비가 내리면서 서울 시내 주요 도로 일부가 통제됐다.', '특히 동부간선도로는 전 구간이 통제돼 출근길 시민들의 불편이 예상된다. . .', '30일 행정안전부에 따르면 오전 8시 현재 서울 용산구 한강 잠수교, 노원구 동부간선도로 등 70개소의 도로가 통제되고 있다.', '북한산과 설악산, 오대산 등 국립공원 3곳 121개 탐방로도 통제가 이뤄지고 있다..', '동부간선도로는 집중호우로 인해 전 구간을 교통 통제 중이다.', '강변북로에서 동부간선도로로 진입하는 지점부터 수락 지하차도까지 양방향이 전면 통제되고 있다.', '경찰은 동부간선도로 통제 해제시간을 이날 낮 12시로 공지했다. .', '전날 밤 9시부터 통제됐던 잠수교는 이날 오전 5시30분부터 차량 통행이 재개됐으나 여전히 보행자 통행은 불가능하다. .', '서울외곽도로 노고산2터널 퇴계원방향 부근에는 토사가 유출돼 복구 중이다.', '편도 4차로 중 3개 차로는 이날 오전 5시45분 복구가 끝나 일부 통행이 재개됐다.. .']


In [ ]:
# kibana 콘솔창에 접속해서 계수 확인
# http://192.168.0.130:5601/app/dev_tools 에 접속해서 해야함

## 입력 ##
# GET korquad/_count

## 출력 ###
'''
{
  "count" : 1420,
  "_shards" : {
    "total" : 2,
    "successful" : 2,
    "skipped" : 0,
    "failed" : 0
}
'''    

In [ ]:
#-------------------------------------------------------------------------------------
# 검색 처리
#-------------------------------------------------------------------------------------
import time
from elasticsearch import Elasticsearch
from myutils import embed_text, divide_arr_avg_exten

def run_query_loop():
    while True:
        try:
            handle_query()
        except KeyboardInterrupt:
            return
        
def handle_query():
    
    query = input("검색어 입력: ")
    
    start_embedding_time = time.time()
    
    # 문장으로 비교할때=>쿼리 문장에 대한 벡터 생성해서 비교
    #query_vector = embed_text(model=bi_encoder,paragraphs=[query])[0]
    #------------------------------------------------------------------
    # 토큰 평균으로 비교할때=> 쿼리 문장에 대한 모든 토큰 벡터를 생성해서 비교
    # 토큰 분할 임베딩 처리
    token_query_embeds = embedding([query])
    
    token_query_embed_arr_list = []
    # 쿼리 문장들의 토큰들의 평균을 구함.
    for token_query_embed in token_query_embeds:
        
        token_query_embed = token_query_embed[1:-1] # 맨앞에 [CLS]와 맨두에 [SEP] 토큰은 뺌
        print(f'*token_query_embed.shape:{token_query_embed.shape}')
        
        tmp = token_query_embed.cpu().numpy().astype('float32')
        tmp=tmp.mean(axis=0) #평균 구함
        
         # Resize 방식으로 차원 축소(384로 줄일대 -2% 성능 저하 발생)
        if DIM_RESIZE_METHOD == 1:
            tmp = np.resize(tmp, (DIM_RESIZE_LEN,))
         # Dense 방식으로 차원 축소=> 평균 구하기 전에 차원 축소하는것이 성능 +0.6더 좋음
        elif DIM_RESIZE_METHOD == 2:
            tmp1 = torch.Tensor([tmp]) # 1차원 배열을 -> 2차둰 텐서(1,768)로 변환
            tmp2 = dense_model(embed_tensor=tmp1, out_f=DIM_RESIZE_LEN, weight=dense_weight, bias=dense_bias)
            tmp = tmp2.detach().numpy().astype('float32').ravel(order='C') # 1차원 배열로 변경(128,)        
        
        token_query_embed_arr_list.append(tmp)
        
    query_vector = np.array(token_query_embed_arr_list)[0]  # 리스트를 배열로 변환  
    
    # print(query_vector)
    print(f'*query_vector.shape:{query_vector.shape}\n')
    #------------------------------------------------------------------
    
    end_embedding_time = time.time() - start_embedding_time
    
    # 쿼리 구성
    '''
    script_query = {
        "script_score":{
            "query":{
                "match_all": {}},
            "script":{
                "source": "cosineSimilarity(params.query_vector, doc['vector2']) + 1.0",  # 뒤에 1.0 은 코사인유사도 측정된 값 + 1.0을 더해준 출력이 나옴(doc['summarize_vector'])
                "params": {"query_vector": query_vector}
            }
        }
    }
    '''
    # 문단별 40개의 벡터와 쿼리벡터를 서로 비교하여 최대값 갖는 문단들중 가장 유사한  문단 출력
    # => script """ 안에 코드는 java 임.
    # => "queryVectorMag": 0.1905 일때 100% 일치하는 값은 9.98임(즉 10점 만점임)
    script_query = {
        "script_score":{
            "query":{
                "match_all": {}
            },
                "script":{
                    "source": """
                      float max_score = 0;
                      for(int i = 1; i <= params.VectorNum; i++) 
                      {
                          float[] v = doc['vector'+i].vectorValue; 
                          float vm = doc['vector'+i].magnitude;  
                          
                          if (v[0] != 0)
                          {
                              float dotProduct = 0;

                              for(int j = 0; j < v.length; j++) 
                              {
                                  dotProduct += v[j] * params.queryVector[j];
                              }

                              float score = dotProduct / (vm * (float) params.queryVectorMag);

                              if(score > max_score) 
                              {
                                  max_score = score;
                              }
                            }
                      }
                      return max_score
                    """,
                "params": 
                {
                  "queryVector": query_vector,
                  "queryVectorMag": 0.1905,
                  "VectorNum": 40
                }
            }
        }
    }
    
    #print('query\n')
    #print(script_query)
    
    # 실제 ES로 검색 쿼리 날림
    start_search_time = time.time()
    response = es.search(
        index=INDEX_NAME,
        body={
            "size": SEARCH_SIZE,
            "query": script_query,
            "_source":{"includes": ["rfile_name","rfile_text"]}
        }
    )
    end_search_time = time.time() - start_search_time
    
    print("{} total hits.".format(response["hits"]["total"]["value"])) 

        
    # 쿼리 응답 결과값에서 _id, _score, _source 등을 뽑아냄
    # print(response)
    texts = []
    titles = [] 
    bi_scores = []
    for hit in response["hits"]["hits"]: 
        '''
        print("index:{}, type:{}".format(hit["_index"], hit["_type"]))
        print("id: {}, score: {}".format(hit["_id"], hit["_score"])) 
        
        print(f'[제목] {hit["_source"]["title"]}')
        
        print('[요약문]')
        print(hit["_source"]["summarize"]) 
        print()
                
        '''
        #print(hit)
        
        # 리스트에 저장해둠
        titles.append(hit["_source"]["rfile_name"])
        texts.append(hit["_source"]["rfile_text"])
        bi_scores.append(hit["_score"])
        
     # 내림 차순으로 정렬
    dec_bi_scores = reversed(np.argsort(bi_scores))
    print(dec_bi_scores)
    
    # 내림차순으로 출력
    for idx in dec_bi_scores:
        print("{:.2f}\t[제목]:{}\n{}\n".format(float(bi_scores[idx]), titles[idx], texts[idx]))
    
    # 처리 시간들 출력
    print("embedding time: {:.2f} ms".format(end_embedding_time * 1000)) 
    print("search time: {:.2f} ms".format(end_search_time * 1000)) 
    print('\n')
    #========================================================================================================    
        

In [ ]:
#====================================================================
# ES 인덱싱된 내용 검색 
# => cosineSimilarity 스크립트를 이용하여 ES로 query 함(*이때 SEARCH_SIZE를 몇개로 할지 지정할수 있음)
# => 쿼리 응답 결과 값에서 _id, _score, _source 등을 뽑아냄
#====================================================================
INDEX_NAME = 'aihub-ts1-acsampe-klue-sbert-v1-mpower10u-128d-onnx-1'  # ES 인덱스 명 (*소문자로만 지정해야 함)
#INDEX_NAME = 'korquad-klue-sbert-v1.0-noavg' # 요약문 평균값 처리 안한경우

SEARCH_SIZE = 5

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.27:9200/")
print(es.info)

# 2. query 처리
run_query_loop()

In [ ]:
#==============================================================================================
# ES index에 데이터 추가하가
# => 추가할 데이터는 {'paragraph': 내용, 'title': 제목} 기존 입려된 방식대로(사전) 입력 되어야 함
#===============================================================================================

# ES에 이미 생성된 index
INDEX_NAME = 'korquad'
BATCH_SIZE = 30


# 1.추가할 데이터 준비
title = [
    '제주도', 
    '한라산',
    '서울특별시'
        ]

paragraph = [
    '대한민국의 남서쪽에 있는 섬. 행정구역상 광역자치단체인 제주특별자치도의 관할. 한국의 섬 중에서 가장 크고 인구가 많은 섬으로 면적은 1833.2㎢이다. 제주도 다음 2번째 큰 섬인 거제도의 5배 정도 된다. 인구는 약 68만 명.',
    '대한민국에서 가장 큰 섬인 제주도에 있으며 대한민국의 실효지배 영토 내의 최고봉이자 가장 높은 산(해발 1,947m). 대한민국의 국립공원 중 하나이다. 국립공원 전역이 유네스코 세계유산으로 지정되었다.',
    '대한민국의 수도인 서울은 현대적인 고층 빌딩, 첨단 기술의 지하철, 대중문화와 예것이 공존하는 대도시. 주목할 만한 명소로는 초현대적 디자인의 컨벤션 홀인 동대문디자인플라자, 한때 7,000여 칸의 방이 자리하던 경복궁, 조계사가 있다',
            ]

# {'paragraph': "", 'title': ""}

# 2. elasticsearch 접속
es = Elasticsearch("http://192.168.0.27:9200/")
print(es.info)

doc = {}
docs = []
count = 0

# 3. batch 사이즈 만큼식 ES에 추가
# => 추가할 데이터는 {'paragraph': 내용, 'title': 제목} 기존 입려된 방식대로(사전) 입력 되어야 함
for title, paragraph in zip(title, paragraph):
    doc = {}
    doc['paragraph'] = paragraph
    doc['title'] = title
    docs.append(doc)
    count += 1
    if count % BATCH_SIZE == 0:
        index_batch(docs)
        docs = []
        print("Indexed {} documents.".format(count))
   
# docs 이 있으면 전송
if docs:
    index_batch(docs)
    print("Indexed {} documents(end).".format(count))


In [ ]:
#==============================================================================================
# ES 데이터 조회하기
#==============================================================================================
INDEX_NAME = 'korquad'

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.27:9200/")
print(es.info)

###########################################################
# 인덱스내 데이터 조회 => query 이용
###########################################################
def search(index, data=None):
    if data is None: #모든 데이터 조회
        data = {"match_all":{}}
    else:
        data = {"match": data}
        
    body = {"query": data}
    res = es.search(index=index, body=body)
    return res
###########################################################

# 모든 데이터 조회
#sr = search(index=INDEX_NAME)
#pprint.pprint(sr)

# 단일 필드 조회
sr = search(index=INDEX_NAME, data = {'title': '제주도'})
print(sr)


In [ ]:
#==============================================================================================
# ES index에 데이터 삭제하기
#==============================================================================================
INDEX_NAME = 'korquad'

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.27:9200/")
print(es.info)

############################################################
## 1: 인덱스 내의 데이터 삭제 => query 이용
############################################################
def delete(index, data):
    if data is None:  # data가 없으면 모두 삭제
        data = {"match_all":{}}
    else:
        data = {"match": data}
        
    body = {"query": data}
    return es.delete_by_query(index, body=body)

############################################################
## 2: 인덱스 내의 데이터 삭제 => id 이용
############################################################
def delete_by_id(index, id):
    return es.delete(index, id=id)

############################################################
## 3: 인덱스 자체 삭제
############################################################
def delete_index(index):
    if es.indices.exists(index=index):
        return es.indices.delete(index=index)


# 1: query 이용 데이터 삭제
delete(index=INDEX_NAME, data={'title':'한라산'})

# 3: 인덱스 자체 삭제
#delete_index(index=INDEX_NAME)


In [ ]:
#==============================================================================================
# ES index에 데이터 업데이트하기
#==============================================================================================
INDEX_NAME = 'korquad'

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.27:9200/")
print(es.info)

############################################################
## 1: 인덱스 내의 데이터 업데이트=>_id 에 데이터 업데이트
############################################################
def update(index, id, doc, doc_type):
    
    body = {
        'doc': doc
    }
    
    res=es.update(index=index, id=id, body=body, doc_type=doc_type)
    return res
############################################################

#=====================================================================
# 검색해서, _id, _type을 구함
sr = search(index=INDEX_NAME, data = {'title': '제주도'})

print('\n')
print("===[검색 결과]===")
for hits in sr["hits"]["hits"]:
    id = hits["_id"]      # id
    type = hits["_type"]  # type
    
    print(f'id: {id}')
    print(f'type: {type}')
    print(f'title:{hits["_source"]["title"]}')
    print(f'paragraph:{hits["_source"]["paragraph"]}')
    print('\n')
    
    # update 시킴
    print("===[업데이트]===")
    doc = {'paragraph': '제주도는 대한민국에 가장 남쪽에 있는 섬으로, 인구는 약 71만명이며, 화산섬으로 관광자원이 많은 천혜의 관광지 이다.'}
    print(doc)
    print('\n')
    
    ur=update(index=INDEX_NAME, id=id, doc=doc, doc_type=type)
    print("===[업데이트 결과]===")
    print(ur)
    print('\n')

#=====================================================================

# 인덱스 refresh 함
# elasticsearch의 자동 새로고침의 시간은 1초 정도 소요
# 따라서 코드에 아래 명령어를 입력하지 않았을 경우 검색을 하지 못할 가능성도 존재
es.indices.refresh(index=INDEX_NAME)

# 제주도로 검색해서 한번더 확인
sr = search(index=INDEX_NAME, data = {'title': '제주도'})

print("===[재검색 결과]===")
for hits in sr["hits"]["hits"]:
    
    print(f'id:{hits["_id"]}')
    print(f'type: {hits["_type"]}')
    print(f'title:{hits["_source"]["title"]}')
    print(f'paragraph:{hits["_source"]["paragraph"]}')
    
              